In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from pytrade.features.structure import FindPeaksTransformer, find_peaks_transformer_func
from pytrade.loaders import fetch_crypto_ohlcv_data
from pytrade.plotting import add_chart_to_figure, create_base_candlestick_chart

In [3]:
# Fetch OHLCV data for BTC/USDT
dfc = fetch_crypto_ohlcv_data(symbol="BTC/USDT", timeframe="1d", lookback_period="3m")

In [4]:
# Make a copy of the data
df = dfc.copy()

In [25]:
# Define parameters for the find_peaks_transformer_func
params = {
    "high_column": "high",
    "low_column": "low",
    "prominence": None,
    "distance": None,
    "width": None,
    "threshold": None,
    "wlen": None,
    "rel_height": None,
    "plateau_size": None,
}

In [26]:
# Apply the find_peaks_transformer_func directly
dfx = find_peaks_transformer_func(df, **params)

In [27]:
# Display the new columns added to the DataFrame
print(dfx["low_local_dips"].dropna().head(3), dfx["high_local_peaks"].dropna().head(3))

timestamp
2025-02-18    93388.09
2025-02-21    94871.95
2025-02-26    82256.01
Name: low_local_dips, dtype: float64 timestamp
2025-02-21    99475.00
2025-03-02    95000.00
2025-03-06    92810.64
Name: high_local_peaks, dtype: float64


In [28]:
# Use the FindPeaksTransformer wrapper
ftr = FindPeaksTransformer(**params)
dfxx = ftr.fit_transform(df)

In [29]:
# Verify that the results from the function and the transformer are identical
dfx.equals(dfxx)

True

In [30]:
# Create a candlestick chart and add the detected peaks and dips
fig = create_base_candlestick_chart(
    df, title=f"peaks: {len(dfx['high_local_peaks'].dropna())}, dips: {len(dfx['low_local_dips'].dropna())}"
)

# Add peaks to the chart
fig = add_chart_to_figure(
    fig,
    go.Scatter(
        x=dfx.index,
        y=dfx["high_local_peaks"],
        mode="markers",
        name="Local Peaks",
        marker={"color": "green", "size": 10},
    ),
)

# Add dips to the chart
fig = add_chart_to_figure(
    fig,
    go.Scatter(
        x=dfx.index,
        y=dfx["low_local_dips"],
        mode="markers",
        name="Local Dips",
        marker={"color": "red", "size": 10},
    ),
)

fig.show()

In [31]:
change_heuristic = np.percentile(df["close"].diff(1).abs().dropna(), 75)
distance_heuristic = int(df.shape[0] * 0.05)

In [32]:
# Define parameters for the find_peaks_transformer_func
params = {
    "high_column": "high",
    "low_column": "low",
    "prominence": change_heuristic,
    "distance": distance_heuristic,
    "width": None,
    "threshold": None,
    "wlen": None,
    "rel_height": None,
    "plateau_size": None,
}

In [33]:
# Apply the find_peaks_transformer_func directly
dfx = find_peaks_transformer_func(df, **params)

In [34]:
# Create a candlestick chart and add the detected peaks and dips
fig = create_base_candlestick_chart(
    df, title=f"peaks: {len(dfx['high_local_peaks'].dropna())}, dips: {len(dfx['low_local_dips'].dropna())}"
)

# Add peaks to the chart
fig = add_chart_to_figure(
    fig,
    go.Scatter(
        x=dfx.index,
        y=dfx["high_local_peaks"],
        mode="markers",
        name="Local Peaks",
        marker={"color": "green", "size": 10},
    ),
)

# Add dips to the chart
fig = add_chart_to_figure(
    fig,
    go.Scatter(
        x=dfx.index,
        y=dfx["low_local_dips"],
        mode="markers",
        name="Local Dips",
        marker={"color": "red", "size": 10},
    ),
)

fig.show()